### https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=test.csv

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import sidetable
%matplotlib inline

df_train = pd.read_csv('./train.csv', index_col='Id')
X_train = df_train.drop(columns='SalePrice')
y_train = df_train['SalePrice']
X_test = pd.read_csv('./test.csv', index_col='Id')
train_test = pd.concat([X_train, X_test])
train_index = X_train.index
test_index = X_test.index

In [2]:
# Have a look at the first five rows
train_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [3]:
threshold = 0.3 * len(train_test)
cols = train_test.isnull().sum()[train_test.isnull().sum() < threshold].index
train_test = train_test[cols]
X_train = train_test.loc[train_index, cols]
X_test = train_test.loc[test_index, cols]

In [4]:
from sklearn.impute import SimpleImputer
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

num_cols = train_test.select_dtypes(include='number').columns
cat_cols = train_test.select_dtypes(exclude='number').columns
si = SimpleImputer(strategy='most_frequent', fill_value='Unknown')
ce = CatBoostEncoder()
ii = IterativeImputer()
sc = StandardScaler()

X_train.drop_duplicates(inplace=True)
X_test.drop_duplicates(inplace=True)

si.fit(train_test[cat_cols])
X_train[cat_cols] = si.transform(X_train[cat_cols])
X_test[cat_cols] = si.transform(X_test[cat_cols])

ce.fit(X_train[cat_cols], y_train)
X_train[cat_cols] = ce.transform(X_train[cat_cols])
X_test[cat_cols] = ce.transform(X_test[cat_cols])

ii.fit(train_test[num_cols])
X_train[num_cols] = ii.transform(X_train[num_cols])
X_test[num_cols] = ii.transform(X_test[num_cols])

sc.fit(train_test[num_cols])
X_train[num_cols] = sc.transform(X_train[num_cols])
X_test[num_cols] = sc.transform(X_test[num_cols])

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)

In [6]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_depth=10, random_state=1, n_estimators = 300).fit(X_train, y_train)
# Return importances of features in descending order
feature_importances = pd.DataFrame(abs(rf.feature_importances_), index =X_train.columns,  columns=['importance']).sort_values('importance', ascending=False)

In [7]:
feature_importances[:10].cumsum()

,importance
OverallQual,0.501237
GrLivArea,0.603308
Neighborhood,0.690858
1stFlrSF,0.726524
TotalBsmtSF,0.760902
BsmtFinSF1,0.787843
GarageCars,0.808267
LotArea,0.826902
GarageArea,0.845462
2ndFlrSF,0.862001


In [8]:
# Take most relevant columns such that 86% of information is kept
cols = feature_importances.iloc[:10].index
X_train = X_train[cols]
X_val = X_val[cols]
X_test = X_test[cols]

In [9]:
X_train

,OverallQual,GrLivArea,Neighborhood,1stFlrSF,TotalBsmtSF,BsmtFinSF1,GarageCars,LotArea,GarageArea,2ndFlrSF
Id,,,,,,,,,,
7,1.355551,0.381924,224868.818344,1.362287,1.439155,2.036246,0.306475,-0.010667,0.757462,-0.785025
808,-0.772552,0.148706,125784.681286,-0.223255,0.617718,1.904532,0.306475,1.422317,0.255971,0.390820
956,-0.063185,0.903700,210053.503767,-0.460321,-0.165144,0.093466,0.306475,-0.384511,0.088808,1.499007
1041,-0.772552,0.597355,137381.509279,1.640139,0.186576,-0.600227,0.306475,0.374971,0.051660,-0.785025
702,0.646183,-0.665580,188940.218863,0.011263,0.254651,-0.969026,0.306475,-0.072044,0.255971,-0.785025
...,...,...,...,...,...,...,...,...,...,...
716,-0.063185,-0.297965,188940.218863,0.485396,0.027735,-0.969026,0.306475,-0.003565,0.023800,-0.785025
906,-0.772552,-0.865199,146002.275203,-0.246197,0.009581,-0.191914,-1.006733,-0.031464,-0.895600,-0.785025
1097,-0.063185,-0.288083,102250.031471,-0.985437,-0.834548,-0.969026,-2.319941,-0.416721,-2.195761,0.572796


In [10]:
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, RepeatedKFold

def objective(trial):
    
    param = {
        'max_depth': trial.suggest_int('max_depth', 10,50),
         'num_leaves': trial.suggest_int('num_leaves', 6, 50), 
         'learning_rate': trial.suggest_categorical('learning_rate', [0.1,0.01,0.001]),
         'min_child_samples': trial.suggest_int('min_child_samples', 100, 500), 
         'min_child_weight': trial.suggest_categorical('min_child_weight', [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4]),
         'subsample': trial.suggest_loguniform('subsample', 0.2, 0.8), 
         'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.4, 0.6),
         'reg_alpha': trial.suggest_categorical('reg_alpha', [0, 1e-1, 1, 2, 5, 7, 10, 50, 100]),
         'reg_lambda': trial.suggest_categorical('reg_lambda', [0, 1e-1, 1, 5, 10, 20, 50, 100])
        }
    model = XGBRegressor(**param)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return (cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_log_error', n_jobs=-1, cv=RepeatedKFold(10))*-1).mean()

In [11]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)
trial = study.best_trial
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-05-11 18:04:48,809] A new study created in memory with name: no-name-0ca0b566-3596-4189-939c-6e0b02f82355
C:\Users\dtngh\miniconda3\envs\gputest\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_splits=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


[18:04:48] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { min_child_samples, num_leaves } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




[I 2021-05-11 18:04:52,265] Trial 0 finished with value: 5.318131519220011 and parameters: {'max_depth': 34, 'num_leaves': 23, 'learning_rate': 0.001, 'min_child_samples': 164, 'min_child_weight': 10000.0, 'subsample': 0.6995146111113196, 'colsample_bytree': 0.4490746472358926, 'reg_alpha': 100, 'reg_lambda': 0}. Best is trial 0 with value: 5.318131519220011.


[18:04:52] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { min_child_samples, num_leaves } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




[I 2021-05-11 18:04:55,404] Trial 1 finished with value: 0.022165497093855898 and parameters: {'max_depth': 35, 'num_leaves': 41, 'learning_rate': 0.1, 'min_child_samples': 199, 'min_child_weight': 1e-05, 'subsample': 0.3614846491871883, 'colsample_bytree': 0.41842415067695576, 'reg_alpha': 5, 'reg_lambda': 20}. Best is trial 1 with value: 0.022165497093855898.


...

[19:03:09] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { min_child_samples, num_leaves } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




[I 2021-05-11 19:03:10,018] Trial 998 finished with value: 0.1629562775271873 and parameters: {'max_depth': 42, 'num_leaves': 41, 'learning_rate': 0.1, 'min_child_samples': 307, 'min_child_weight': 10000.0, 'subsample': 0.7614959603831629, 'colsample_bytree': 0.4279409971098741, 'reg_alpha': 10, 'reg_lambda': 20}. Best is trial 448 with value: 0.021471381896315766.


[19:03:10] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { min_child_samples, num_leaves } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




[I 2021-05-11 19:03:12,913] Trial 999 finished with value: 0.022225066232641977 and parameters: {'max_depth': 50, 'num_leaves': 10, 'learning_rate': 0.1, 'min_child_samples': 318, 'min_child_weight': 10.0, 'subsample': 0.7973703234253967, 'colsample_bytree': 0.43110141348795655, 'reg_alpha': 10, 'reg_lambda': 50}. Best is trial 448 with value: 0.021471381896315766.


Accuracy: 0.021471381896315766
Best hyperparameters: {'max_depth': 46, 'num_leaves': 40, 'learning_rate': 0.1, 'min_child_samples': 346, 'min_child_weight': 10.0, 'subsample': 0.7158297501972162, 'colsample_bytree': 0.46788482382872076, 'reg_alpha': 0.1, 'reg_lambda': 20}


In [12]:
from yellowbrick.model_selection import learning_curve

# Build model with the best params
best = XGBRegressor(**study.best_params )

In [14]:
best.fit(X_train, y_train)
y_pred = best.predict(X_test)
y_pred = y_pred.astype(int)
y_pred = pd.DataFrame(y_pred, columns=['SalePrice'], index=X_test.index)
# saving the dataframe 
y_pred.to_csv('Predictions.csv') 

[19:23:34] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { min_child_samples, num_leaves } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


